# Cell counter interface

This notebook aims to provide a streamlined interface for cell counting.

In [14]:
from ipywidgets import widgets, fixed
from skimage import io, exposure, morphology
import numpy as np
%matplotlib inline

In [3]:
uploaded_file = widgets.FileUpload(multiple=False, description="Upload")
uf_name = widgets.Text(value=None,
                       placeholder='no file loaded',
                       description='',
                       disabled=True
                      )
channel_selector = widgets.Dropdown(options=['(no file loaded)'],
                                            value='(no file loaded)',
                                            description='Channel:',
                                            disabled=True
                                           )
output = widgets.Output() #only used for debugging
#output_image = widgets.Image(value=b'',format='png',width='300',height='400')


def new_file_uploaded(change):
    new_name = list(change['new'])[0]
    new_img = io.imread(change['new'][new_name]['content'], plugin='imageio')
    channels = [str(i+1) for i in range(new_img.shape[-1])]
    channel_selector.options = channels
    channel_selector.disabled = False
    with output:
        print(io.imread(change['new'][new_name]['content'], plugin='imageio').shape)
        print([str(i+1) for i in range(new_img.shape[-1])])
    uf_name.value = new_name

uploaded_file.observe(new_file_uploaded, 'value')
widgets.HBox([uploaded_file,
              uf_name,
              #output,
              channel_selector])

In [4]:
def adjust_image(image,
                 lower_thresh=2, upper_thresh=98,
                 filter_size=0, 
                 cmap='jet'
                ):
    """
    Applies contrast stretching to an image, then
    uses a white top-hat filter to remove small patches
    of brightness that would cause false positives later.
    
    Input: image; values for min and max percentile
    brightnesses to keep; size below which bright patches
    will be removed; colourmap.
    
    Output: image, hopefully with most of the background stripped
    out. If it's worked well, it'll look like bright blobs on a
    dark background.
    """
    
    p2, p98 = np.percentile(image, (lower_thresh, upper_thresh))
    img_rescale = exposure.rescale_intensity(image, in_range=(p2, p98))
    selem = morphology.disk(filter_size)
    wht_tophat = morphology.white_tophat(img_rescale,selem=selem)
    io.imshow(img_rescale - wht_tophat, cmap=cmap)
    
    return img_rescale

In [37]:
widget_style = {'description_width': 'initial'}
lower_thresh = widgets.IntSlider(min=0, max=100, step=5, value=2,
                                 description='lower threshold', style=widget_style)
upper_thresh = widgets.IntSlider(min=0, max=100, step=5, value=98,
                                 description='upper threshold', style=widget_style)
filter_size  = widgets.IntSlider(min=0, max=20, step=1, value=0,
                                 description='filter size', style=widget_style)

contrast_ui = widgets.VBox([lower_thresh, upper_thresh, filter_size])
contrast_tweaking = widgets.interactive_output(adjust_image, {'lower_thresh': lower_thresh,
                                                              'upper_thresh': upper_thresh,
                                                              'filter_size': filter_size,
                                                              'cmap': fixed('viridis'),
                                                              'image': fixed(io.imread('Drd2_Adult_S_conf_25X_CPU1_cryo_NAV.jpg')[:,:,int(channel_selector.value)-1])})

display(contrast_ui, contrast_tweaking)

Output()